## Precición de calvicie 

El IA Project 2 consiste en realizar desplegar un modelo en streamlit. Nuestro equipo a elegido el dataset [Hair Health Prediction](https://www.kaggle.com/datasets/amitvkulkarni/hair-health) para predecir si se te está cayendo el pelo

## Imports y acercamiento al dataset

In [ ]:
!which python

: 

In [ ]:
# Load dependencies for loading data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import dependencies for pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

: 

In [ ]:
data = pd.read_csv('data/Predict Hair Fall.csv')

### Factors Contributing to Baldness

1. **Genetics**:  
   Indicates whether the individual has a family history of baldness (Yes/No).

2. **Hormonal Changes**:  
   Indicates whether the individual has experienced hormonal changes (Yes/No).

3. **Medical Conditions**:  
   Lists specific medical conditions that may contribute to baldness, such as:
   - Alopecia Areata  
   - Thyroid Problems  
   - Scalp Infection  
   - Psoriasis  
   - Dermatitis  

4. **Medications & Treatments**:  
   Lists medications and treatments that may lead to hair loss, including:
   - Chemotherapy  
   - Heart Medication  
   - Antidepressants  
   - Steroids  

5. **Nutritional Deficiencies**:  
   Lists nutritional deficiencies that may contribute to hair loss, such as:
   - Iron deficiency  
   - Vitamin D deficiency  
   - Biotin deficiency  
   - Omega-3 fatty acid deficiency  

6. **Stress**:  
   Indicates the stress level of the individual (Low/Moderate/High).

7. **Age**:  
   Represents the age of the individual.

8. **Poor Hair Care Habits**:  
   Indicates whether the individual practices poor hair care habits (Yes/No).

9. **Environmental Factors**:  
   Indicates whether the individual is exposed to environmental factors that may contribute to hair loss (Yes/No).

10. **Smoking**:  
    Indicates whether the individual smokes (Yes/No).

11. **Weight Loss**:  
    Indicates whether the individual has experienced significant weight loss (Yes/No).

12. **Baldness (Target)**:  
    Binary variable indicating the presence (1) or absence (0) of baldness in the individual.


In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data

## Análisis de variables

### Variables categóricas

In [ ]:
categorical_columns = data.select_dtypes(include='object').columns
categorical_columns

In [ ]:
sns.countplot(data['Genetics'])

In [ ]:
sns.countplot(data['Hormonal Changes'])

In [ ]:
sns.countplot(data["Medical Conditions"])

In [ ]:
data.loc[data["Medical Conditions"] == "No Data", "Medical Conditions"] = "No CondMedicas"

In [ ]:
sns.countplot(data["Medications & Treatments"])

In [ ]:
data.loc[data["Medications & Treatments"] == "No Data", "Medications & Treatments"] = "No Medications"

In [ ]:
sns.countplot(data["Nutritional Deficiencies "])


In [ ]:
data.loc[data["Nutritional Deficiencies "] == "No Data", "Nutritional Deficiencies "] = "No Deficiencies"

In [ ]:
sns.countplot(data["Stress"])

In [ ]:
sns.countplot(data["Poor Hair Care Habits "])

In [ ]:
sns.countplot(data["Environmental Factors"])

In [ ]:
sns.countplot(data["Smoking"])

In [ ]:
sns.countplot(data["Weight Loss "])


In [ ]:
sns.boxplot(data['Age'])

In [ ]:
label_counts = data["Hair Loss"].value_counts()

plt.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette("Set2", len(label_counts)))
plt.title('0: no presencia de caida, 1: caida')

# Mostrar el gráfico
plt.show()

# Mostrar los conteos de cada etiqueta
print(label_counts)

## Preprocessing Data

Vamos a entrenar una red neuronal. Por tanto, vamos a tratar los datos para ello. No hará falta una ingeniería de características porque es la propia red la que se encarga de ello. 

Las transformcaiones a hacer son las siguientes: 
- Genetics, hormonal changes, Poor Hair Care Habits, Environmental Factors, Smoking, Weight Loss: y/n to 0/1

- Medical Conditions, Medications & Treatments, Nutritional Deficiencies: one hot encoding 

- Stress: moderate, high, low -> 0,1,2

In [ ]:
map_yes_no = {
    "Yes": 1,
    "No": 0
}

In [ ]:
data.columns

In [ ]:
colums_yes_no = ["Genetics" ,"Hormonal Changes", "Poor Hair Care Habits ", "Environmental Factors", "Smoking", "Weight Loss "]

In [ ]:
columns_onehotencoder = ["Medical Conditions", "Medications & Treatments", "Nutritional Deficiencies "]

In [ ]:
# Iterar por las columnas y mapear los valores
for col in colums_yes_no:
    if col in data.columns:  # Verifica que la columna exista
        data[col] = data[col].map(map_yes_no)
    else:
        print(f"Columna {col} no encontrada en el dataframe.")

In [ ]:
data

In [ ]:
# Instanciar el codificador
encoder = OneHotEncoder()  # sparse=False para obtener una matriz densa
for col in columns_onehotencoder:
    
    # Ajustar y transformar la columna 'col'
    encoded = encoder.fit_transform(data[[col]]).toarray()
    # Convertir el resultado en un DataFrame
    encoded_df = pd.DataFrame(encoded, columns=encoder.categories_[0])

    # Concatenar el DataFrame original con las columnas codificadas
    data = pd.concat([data, encoded_df], axis=1)
    
    data.drop(col, axis=1, inplace=True)

print(data)


In [ ]:
data

In [ ]:
map_stres = {
    "Low": 0,
    "Moderate": 1,
    "High": 2
}
data["Stress"] = data["Stress"].map(map_stres)

In [ ]:
data.drop("Id", axis = 1, inplace = True)

In [ ]:
data

## Train ML

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from scipy import stats

In [ ]:
# Separate input variables from output label
X = data.drop("Hair Loss", axis=1)
y = data["Hair Loss"]

In [ ]:
# Normalizing the data to improve stability while training
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [ ]:
# Split data in training and validation partitions
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2, stratify=y)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,
                                                  test_size=0.5, stratify=y_val)

In [ ]:
# Show sizes of partitions
print("Size of training data: ", X_train.shape)
print("Size of training labels: ", y_train.shape)
print("Size of validation data: ", X_val.shape)
print("Size of validation labels: ", y_val.shape)
print("Size of test labels: ", X_test.shape)
print("Size of test labels: ", y_test.shape)

In [ ]:
RF = RandomForestClassifier()
f1_scorer = cross_val_score(RF, X_train, y_train, cv=5)
media_f1 = f1_scorer.mean()
# Intervalo de confianza con nivel de confianza 95%
ci_95 = stats.t.interval(0.95, len(f1_scorer)-1, loc=media_f1, scale=stats.sem(f1_scorer))

# Imprimir resultados
print(f"F1-Score medio: {media_f1:.4f}")
print(f"Intervalo de confianza del 95%: ({ci_95[0]:.4f}, {ci_95[1]:.4f})")

In [ ]:
sv = SVC()
f1_scorer = cross_val_score(sv, X_train, y_train, cv=5)
media_f1 = f1_scorer.mean()
# Intervalo de confianza con nivel de confianza 95%
ci_95 = stats.t.interval(0.95, len(f1_scorer)-1, loc=media_f1, scale=stats.sem(f1_scorer))

# Imprimir resultados
print(f"F1-Score medio: {media_f1:.4f}")
print(f"Intervalo de confianza del 95%: ({ci_95[0]:.4f}, {ci_95[1]:.4f})")

In [ ]:
gb = GradientBoostingClassifier()
f1_scorer = cross_val_score(gb, X_train, y_train, cv=5)
media_f1 = f1_scorer.mean()
# Intervalo de confianza con nivel de confianza 95%
ci_95 = stats.t.interval(0.95, len(f1_scorer)-1, loc=media_f1, scale=stats.sem(f1_scorer))

# Imprimir resultados
print(f"F1-Score medio: {media_f1:.4f}")
print(f"Intervalo de confianza del 95%: ({ci_95[0]:.4f}, {ci_95[1]:.4f})")

## Experimentación 

Partimos de las métricas de modelos base procedemos a realizar experimentos a ver si mejoran los resultados. 


In [ ]:
def evaluate_models(data, target_column = 'Hair Loss'):
    """
    Separa las variables de entrada y salida, normaliza los datos, divide en conjuntos de entrenamiento, validación y prueba,
    y evalúa tres modelos diferentes (RandomForest, SVC, GradientBoostingClassifier) con validación cruzada.
    
    Parámetros:
    data (pd.DataFrame): DataFrame de entrada.
    target_column (str): Nombre de la columna objetivo.
    
    Retorna:
    dict: Diccionario con los F1-Scores medios y sus intervalos de confianza del 95% para cada modelo.
    """
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    sc = StandardScaler()
    X = sc.fit_transform(X)
    
    # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
    # X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, stratify=y_val)
    
    models = {
        "RandomForest": RandomForestClassifier(),
        "SupportVectorMachine": SVC(),
        "GradientBoosting": GradientBoostingClassifier()
    }
    
    results = {}
    
    for name, model in models.items():
        # f1_scorer = cross_val_score(model, X_train, y_train, cv=5)
        f1_scorer = cross_val_score(model, X, y, cv=5)
        media_f1 = f1_scorer.mean()
        ci_95 = stats.t.interval(0.95, len(f1_scorer)-1, loc=media_f1, scale=stats.sem(f1_scorer))
        
        results[name] = {
            "F1-Score medio": round(media_f1, 4),
            "Intervalo de confianza 95%": (round(ci_95[0], 4), round(ci_95[1], 4))
        }
        
        print(f"{name} - F1-Score medio: {media_f1:.4f}")
        print(f"{name} - Intervalo de confianza del 95%: ({ci_95[0]:.4f}, {ci_95[1]:.4f})")
    
    return results


### Procesamiento básico para para partir hacia todas las experimentaciones. 

In [ ]:
data = pd.read_csv('data/Predict Hair Fall.csv')

## Mapero de Si y No
map_yes_no = {
    "Yes": 1,
    "No": 0
}
colums_yes_no = ["Genetics" ,"Hormonal Changes", "Poor Hair Care Habits ", "Environmental Factors", "Smoking", "Weight Loss "]

# Iterar por las columnas y mapear los valores
for col in colums_yes_no:
    if col in data.columns:  # Verifica que la columna exista
        data[col] = data[col].map(map_yes_no)
    else:
        print(f"Columna {col} no encontrada en el dataframe.")

# Mapeo estrés
map_stres = {
    "Low": 0,
    "Moderate": 1,
    "High": 2
}
data["Stress"] = data["Stress"].map(map_stres)

# Eliminar ID: 
data.drop("Id", axis = 1, inplace = True)

data

### Experimento 1

Agrupación de columnas 

In [ ]:
def asignar_grupo(diccionario):
    """
    Devuelve una función que asigna un grupo basado en el valor encontrado en el diccionario.
    """
    def asignar(valor):
        for grupo, valores in diccionario.items():
            if valor in valores:
                return grupo
        return "Otros"  # Por seguridad
    return asignar

In [ ]:
def one_hot_encode(data, columns):
    """
    Aplica One-Hot Encoding a las columnas especificadas de un DataFrame.
    
    Parámetros:
    data (pd.DataFrame): DataFrame de entrada.
    columns (list): Lista de nombres de columnas a codificar.
    
    Retorna:
    pd.DataFrame: DataFrame con las columnas codificadas y eliminadas las originales.
    """
    encoder = OneHotEncoder()
    
    for col in columns:
        encoded = encoder.fit_transform(data[[col]]).toarray()
        encoded_df = pd.DataFrame(encoded, columns=encoder.categories_[0])
        
        data = pd.concat([data, encoded_df], axis=1)
        data.drop(col, axis=1, inplace=True)
    
    return data

#### Agrupación de Medical Conditions

In [ ]:
exp11 = data.copy()
exp11["Medical Conditions"] = exp11["Medical Conditions"].str.strip()

In [ ]:
grupos_condiciones = {
        "Enfermedades inflamatorias de la piel": ["Eczema", "Psoriasis", "Dermatitis", "Seborrheic Dermatitis"],
        "Infecciones": ["Ringworm", "Scalp Infection"],
        "Trastornos del cabello": ["Alopecia Areata", "Androgenetic Alopecia"],
        "Problemas sistémicos": ["Thyroid Problems"],
        "Término genérico": ["Dermatosis"],
        "No Data": ["No Data"]
    }

In [ ]:
exp11["Grupo_Condiciones"] = exp11["Medical Conditions"].apply(asignar_grupo(grupos_condiciones))
exp11.drop("Medical Conditions",axis=1,inplace=True)

In [ ]:
columns_onehotencoder = ["Medications & Treatments", "Nutritional Deficiencies ", "Grupo_Condiciones"]

In [ ]:
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
exp11

In [ ]:
result = evaluate_models(exp11)

Esta agrupación parece mejorar los resultados de los modelos base

#### Agrupación de Deficiendias

In [ ]:
exp11 = data.copy()
exp11["Nutritional Deficiencies "] = exp11["Nutritional Deficiencies "].str.strip()

In [ ]:
grupos_deficiencias = {
        "Vitaminas": ["Vitamin A Deficiency", "Vitamin D Deficiency", "Biotin Deficiency", "Vitamin E deficiency"],
        "Minerales": ["Magnesium deficiency", "Selenium deficiency", "Zinc Deficiency"],
        "Macronutrientes": ["Protein deficiency"],
        "Ácidos grasos": ["Omega-3 fatty acids"],
        "Sin deficiencia": ["No Data", "Iron deficiency"]
    }

In [ ]:
exp11["Grupo Deficiencias"] = exp11["Nutritional Deficiencies "].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Nutritional Deficiencies ",axis=1,inplace=True)
columns_onehotencoder = ["Medications & Treatments", "Grupo Deficiencias", "Medical Conditions"]

In [ ]:
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
exp11

In [ ]:
result = evaluate_models(exp11)

Parece que esta agrupación puede mejorar las predicciones del modelo

#### Agrupación de Deficiendias

In [ ]:
exp11 = data.copy()
exp11["Medications & Treatments"] = exp11["Medications & Treatments"].str.strip()

In [ ]:
grupos_medicamentos = {
        "Antibióticos/Antifúngicos": ["Antibiotics", "Antifungal Cream"],
        "Enfermedades crónicas": ["Blood Pressure Medication", "Heart Medication"],
        "Inmunológicos": ["Immunomodulators", "Steroids"],
        "Salud mental": ["Antidepressants"],
        "Tratamientos para caída de cabello": ["Rogaine", "Accutane"],
        "Quimioterapia": ["Chemotherapy"],
        "No Data": ["No Data"]
    }

In [ ]:
exp11["Grupo Medicamentos"] = exp11["Medications & Treatments"].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Medications & Treatments",axis=1,inplace=True)
columns_onehotencoder = ["Grupo Medicamentos", "Nutritional Deficiencies ", "Medical Conditions"]

In [ ]:
exp11

In [ ]:
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
result = evaluate_models(exp11)

Parece mejorar la predicción del modelo

#### Todas las agrupaciones

In [ ]:
exp11 = data.copy()
exp11["Medical Conditions"] = exp11["Medical Conditions"].str.strip()
exp11["Grupo_Condiciones"] = exp11["Medical Conditions"].apply(asignar_grupo(grupos_condiciones))
exp11.drop("Medical Conditions",axis=1,inplace=True)
exp11["Nutritional Deficiencies "] = exp11["Nutritional Deficiencies "].str.strip()
exp11["Grupo Deficiencias"] = exp11["Nutritional Deficiencies "].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Nutritional Deficiencies ",axis=1,inplace=True)
exp11["Medications & Treatments"] = exp11["Medications & Treatments"].str.strip()
exp11["Grupo Medicamentos"] = exp11["Medications & Treatments"].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Medications & Treatments",axis=1,inplace=True)

In [ ]:
columns_onehotencoder = ["Grupo_Condiciones", "Grupo Deficiencias", "Grupo Medicamentos"]
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
result = evaluate_models(exp11)

### Experimento 2
Agrupar la edad

In [ ]:
exp11 = data.copy()

In [ ]:
# Crear los bins para los grupos de edad
bins = [0, 25, 35, 45, 60]

# Definir las etiquetas para cada grupo
labels = [0, 1, 2, 3]

# Crear la nueva columna con los grupos de edad
exp11['Age_group'] = pd.cut(exp11['Age'], bins=bins, labels=labels, right=False)


exp11.drop("Age", axis=1, inplace=True)

In [ ]:
columns_onehotencoder = ["Medical Conditions", "Nutritional Deficiencies ", "Medications & Treatments"]
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
exp11

In [ ]:
result = evaluate_models(exp11)

### Toda la experimentación junta

In [ ]:
exp11 = data.copy()
exp11["Medical Conditions"] = exp11["Medical Conditions"].str.strip()
exp11["Grupo_Condiciones"] = exp11["Medical Conditions"].apply(asignar_grupo(grupos_condiciones))
exp11.drop("Medical Conditions",axis=1,inplace=True)
exp11["Nutritional Deficiencies "] = exp11["Nutritional Deficiencies "].str.strip()
exp11["Grupo Deficiencias"] = exp11["Nutritional Deficiencies "].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Nutritional Deficiencies ",axis=1,inplace=True)
exp11["Medications & Treatments"] = exp11["Medications & Treatments"].str.strip()
exp11["Grupo Medicamentos"] = exp11["Medications & Treatments"].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Medications & Treatments",axis=1,inplace=True)

In [ ]:
# Crear los bins para los grupos de edad
bins = [0, 25, 35, 45, 60]

# Definir las etiquetas para cada grupo
labels = [0, 1, 2, 3]

# Crear la nueva columna con los grupos de edad
exp11['Age_group'] = pd.cut(exp11['Age'], bins=bins, labels=labels, right=False)


exp11.drop("Age", axis=1, inplace=True)

In [ ]:
columns_onehotencoder = ["Grupo_Condiciones", "Grupo Deficiencias", "Grupo Medicamentos"]
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [ ]:
result = evaluate_models(exp11)

## Optimización del modelo

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
X = exp11.drop("Hair Loss", axis=1)
y = exp11["Hair Loss"]
    
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
gb = GradientBoostingClassifier()

param_grid = {
    "n_estimators": [50, 100, 200],  # Número de árboles en el ensamble
    "learning_rate": [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    "max_depth": [3, 5, 7],  # Profundidad máxima de los árboles
    "subsample": [0.8, 1.0],  # Proporción de muestras utilizadas para entrenar cada árbol
    "min_samples_split": [2, 5, 10],  # Mínimo de muestras para dividir un nodo
}

# Configurar el Grid Search
grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    scoring='f1_weighted',  # Usar 'f1_weighted' para manejar múltiples clases
    cv=3,  # Validación cruzada con 5 pliegues
    verbose=1
)

In [ ]:
grid_search.fit(X, y)

# Imprimir los resultados
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación F1 (ponderada) en entrenamiento:", grid_search.best_score_)

In [ ]:
# Crear un modelo base SVM
svm_model = SVC()

# Definir los parámetros para Grid Search
param_grid = {
    "C": [0.1, 1, 10],  # Controla la penalización del margen de error
    "kernel": ["linear", "rbf", "poly"],  # Tipos de kernel disponibles
    "gamma": ["scale", "auto", 0.01, 0.1],  # Solo aplicable a 'rbf' y 'poly'
    "degree": [2, 3],  # Solo aplicable a 'poly'
}

# Configurar el Grid Search
grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    scoring='f1',  # Usar 'f1_weighted' para manejar múltiples clases
    cv=3,  # Validación cruzada con 5 pliegues
    verbose=1
)

In [ ]:
grid_search.fit(X, y)

# Imprimir los resultados
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación F1 (ponderada) en entrenamiento:", grid_search.best_score_)


In [ ]:
rf = RandomForestClassifier()

param_grid = {
    "n_estimators": [50, 100, 200],  # Número de árboles en el bosque
    "max_depth": [None, 10, 20],  # Profundidad máxima de los árboles
    "min_samples_split": [2, 5, 10],  # Mínimo de muestras para dividir un nodo
    "min_samples_leaf": [1, 2, 4],  # Mínimo de muestras en una hoja
    "max_features": ["sqrt", "log2"],  # Número de características consideradas en cada división
    "bootstrap": [True, False],  # Si se usa muestreo con reemplazo
}


# Configurar el Grid Search
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='f1',  # Usar 'f1_weighted' para manejar múltiples clases
    cv=3,  # Validación cruzada con 5 pliegues
    verbose=1
)

In [ ]:
grid_search.fit(X, y)

# Imprimir los resultados
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación F1 (ponderada) en entrenamiento:", grid_search.best_score_)

## Staking de modelos. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [ ]:
base_models = [
    ('gradient_boost', GradientBoostingClassifier(
        learning_rate=0.2, max_depth=7, min_samples_split=2, 
        n_estimators=100, subsample=0.8)),
    
    ('svc', SVC(C=10, degree=3, gamma=0.1, kernel='poly', probability=True)),  # `probability=True` es necesario para Stacking
    
    ('random_forest', RandomForestClassifier(
        bootstrap=True, max_depth=20, max_features='log2', 
        min_samples_leaf=1, min_samples_split=5, n_estimators=100))
]

# Meta-modelo (usamos regresión logística para combinar predicciones)
meta_model = LogisticRegression()

# Definir el StackingClassifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

# Entrenar el modelo con los datos de entrenamiento
stacking_clf.fit(X_train, y_train)

# Evaluar en el conjunto de validación
y_pred = stacking_clf.predict(X_val)

# Calcular métricas de desempeño
from sklearn.metrics import f1_score, accuracy_score

f1 = f1_score(y_val, y_pred)
accuracy = accuracy_score(y_val, y_pred)

print(f"Stacking - F1 Score: {f1:.4f}")
print(f"Stacking - Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def optimized_stacking(X, y):
    """
    Realiza un Grid Search sobre los modelos base de un StackingClassifier y entrena el mejor modelo.

    Parámetros:
    X (array): Características de entrada.
    y (array): Variable objetivo.

    Retorna:
    dict: Resultados del mejor modelo de stacking con F1-Score y Accuracy.
    """

    # Dividir datos en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

    # Definir los clasificadores base con opciones de hiperparámetros
    base_estimators = {
        "gradient_boost": GradientBoostingClassifier(),
        "svc": SVC(probability=True),
        "random_forest": RandomForestClassifier()
    }

    param_grid = {
        "gradient_boost__n_estimators": [50, 100], 
        "gradient_boost__learning_rate": [0.1, 0.2],  
        "gradient_boost__max_depth": [5, 7],  
        
        "svc__C": [1, 10],  
        "svc__kernel": ["poly", "rbf"],  
        "svc__gamma": [0.1, "scale"],  
        
        "random_forest__n_estimators": [50, 100],  
        "random_forest__max_depth": [10, 20],  
        "random_forest__min_samples_split": [2, 5]
    }

    # Definir el StackingClassifier
    stacking_clf = StackingClassifier(
        estimators=[(name, clf) for name, clf in base_estimators.items()],
        final_estimator=LogisticRegression(),
        cv=3, n_jobs=-1
    )

    # Hacer Grid Search sobre los hiperparámetros de los modelos base
    grid_search = GridSearchCV(
        stacking_clf, param_grid, cv=3, scoring="f1", n_jobs=-1, verbose=2
    )

    # Ajustar el modelo
    grid_search.fit(X_train, y_train)

    # Evaluar el mejor modelo encontrado
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)

    f1 = f1_score(y_val, y_pred)
    accuracy = accuracy_score(y_val, y_pred)

    print(f"Mejores parámetros: {grid_search.best_params_}")
    print(f"Stacking - F1 Score: {f1:.4f}")
    print(f"Stacking - Accuracy: {accuracy:.4f}")

    return {
        "Mejores parámetros": grid_search.best_params_,
        "F1 Score": round(f1, 4),
        "Accuracy": round(accuracy, 4)
    }

# Llamar a la función con X e y
resultados = optimized_stacking(X, y)


## Conclusiones

Vemos que el mejor modelo es el ...